## Give a name to the Project and Import the dataset from the URL.

In [ ]:
ProjectName = "Generating Anime Character Faces Using ACGANs"

!pip install opendatasets --upgrade --quiet

In [ ]:
import opendatasets as od

dataset = 'https://www.kaggle.com/splcher/animefacedataset'
od.download(dataset)

import os

Data_Dir = './animefacedataset'
print(os.listdir(Data_Dir + '/images')[:25])

## Pre-process (Normalize) the Training dataset

In [ ]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
import torch
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
%matplotlib inline

# resize training data to be of size 64x64
ImageSize = 64

# send images in the batch size of 128
BatchSize = 128

# Set the parameters to normalize the image in range of -1,1
stat = (0.5, 0.5, 0.5), (0.5, 0.5, 0.5)

# Normalize the entire dataset to avoid weight normalities or abnorla behaviour of the dataset
trainDS = ImageFolder(Data_Dir, transform=T.Compose([T.Resize(ImageSize), T.CenterCrop(ImageSize), T.ToTensor(), T.Normalize(*stat)]))

# Normalize the data containing faces 
trainDL = DataLoader(trainDS, BatchSize, shuffle=True, num_workers=3, pin_memory=True)

# helper method to denormalize the tensors of images and display some smaples from the data
def denorm(ImgTensors):
    return ImgTensors * stat[1][0] + stat[0][0]

# helper method to display the samples images after denormalizing them
def ShowImages(image, nmax=64):
    figure, axis = plt.subplots(figsize=(9, 9))
    axis.set_xticks([]); axis.set_yticks([])
    axis.imshow(make_grid(denorm(image.detach()[:nmax]), nrow=8).permute(1, 2, 0))

# Helper method to plot denormalized images in from the given batches of data
def ShowBatch(DL, nmax=32):
    for image, _ in DL:
        ShowImages(image, nmax)
        break

# call the function
ShowBatch(trainDL)

## Utility Function to Use the GPU/ CUDA on N-VIDIA 

In [ ]:
# utility function to pick GPU is available, else select CPU
def Get_Default_Device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

# utility function to move tensors to the chosen device from previous state
def To_Device(Data, Device):
    if isinstance(Data, (list,tuple)):
        return [To_Device(x, Device) for x in Data]
    return Data.to(Device, non_blocking=True)

# Driver Class to wrap a dataloader to move to the selected device
class Device_Data_Loader():
    def __init__(self, Dl, Device):
        self.Dl = Dl
        self.Device = Device
        
    def __iter__(self):
        for i in self.Dl: 
            yield To_Device(i, self.Device)
    
    # return the number of batches
    def __len__(self):
        return len(self.Dl)

Device = Get_Default_Device()
Device


trainDL = Device_Data_Loader(trainDL, Device)

## Discriminator Network Architecture

In [ ]:
import torch.nn as nn

# design the discriminator architecture
Disc = nn.Sequential(
    
    # first layer has input 3*64*64 and output of 64*32*32
    nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.LeakyReLU(0.2, inplace=True),

    # second layer has input same as the output of previous layer and its own output of 128*16*16
    nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.LeakyReLU(0.2, inplace=True),

    # Third hidden layer has an output of 256*8*8
    nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.LeakyReLU(0.2, inplace=True),

    # fourth hidden layer has an output of 512*4*4 
    nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(512),
    nn.LeakyReLU(0.2, inplace=True),

    # last output layer has output of 1*1*1
    nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0, bias=False),

    # At last, we use a sigmoid activation function as it is a binary classification problem and a flatten layer to reshape the output
    nn.Flatten(),
    nn.Sigmoid())

Disc = To_Device(Disc, Device)

## Design the Discriminator network architecture

In [ ]:
LatentSize = 128
Gene = nn.Sequential(
    
    # Give the input of latent size and 1*1 and output a size of 512*4*4
    nn.ConvTranspose2d(LatentSize, 512, kernel_size=4, stride=1, padding=0, bias=False),
    nn.BatchNorm2d(512),
    nn.ReLU(True),

    # input from previous layer and output a kernel of size 256*8*8
    nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.ReLU(True),

    # input from previous layer and output of 128*16*16
    nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.ReLU(True),

    # input from previous layer and output of size 64*32*32
    nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(True),

    # input from previou layer and final output of 3*64*64
    nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=False),
    nn.Tanh()
    # out: 3 x 64 x 64
)

# Generating random tensor vecotrs in the given size
Ran_Tensor = torch.randn(BatchSize, LatentSize, 1, 1) 

# Now using the above network to generate fake samples
FakeImages = Gene(Ran_Tensor)
print(FakeImages.shape)

# Show the generated fake image
ShowImages(FakeImages)

# choose GPu for generator network if available
Gene = To_Device(Gene, Device)

## Training Discriminator and Generator Networks

In [ ]:
# Function to train discriminator
def Train_Disc(Real_Images, Opt_D):
    
    # clear the discimininator gradients after evey epoch
    Opt_D.zero_grad()

    # In first iteration, we pass the real images and compute the loss setting the target labels to 1
    Real_Preds = Disc(Real_Images)

    Real_Targets = torch.ones(Real_Images.size(0), 1, device=Device)
    
    Real_Loss = F.binary_cross_entropy(Real_Preds, Real_Targets)
    
    Real_Score = torch.mean(Real_Preds).item()
    
    # Generate synthetic (fake) images using generator network
    Latent = torch.randn(BatchSize, LatentSize, 1, 1, device=Device)
    
    Fake_Images = Gene(Latent)

    # Compute the loss by passing images throught he disc. network
    Fake_Targets = torch.zeros(Fake_Images.size(0), 1, device=Device)

    Fake_Preds = Disc(Fake_Images)
    
    Fake_Loss = F.binary_cross_entropy(Fake_Preds, Fake_Targets)
    
    Fake_Score = torch.mean(Fake_Preds).item()

    # Keep updating the weights of the disc. network
    Loss = Real_Loss + Fake_Loss

    Loss.backward()

    Opt_D.step()
    
    return Loss.item(), Real_Score, Fake_Score


# Function to train the generator network
def Train_Gene(Opt_G):
    
    # clearning the generator gradients every epoch
    Opt_G.zero_grad()
    
    # Generate synthetic (fake) images 
    Latent = torch.randn(BatchSize, LatentSize, 1, 1, device=Device)
    
    Fake_Images = Gene(Latent)
    
    # step to give the output to the deiscriminator to compute minimal loss
    Preds = Disc(Fake_Images)

    Targets = torch.ones(BatchSize, 1, device=Device)
    
    Loss = F.binary_cross_entropy(Preds, Targets)
    
    # Keep updating the weights of the generator network
    Loss.backward()

    Opt_G.step()
    
    return Loss.item()

## Function to save the output generated Images

In [ ]:
# saving the images generator by generator network
from torchvision.utils import save_image

Sample_Dir = 'Generated Images'

os.makedirs(Sample_Dir, exist_ok=True)

# utility function
def Save_Samples(Index, Latent_Tensors, show=True):
    
    Fake_Images = Gene(Latent_Tensors)
    
    Fake_Fname = 'Generated_Images-{0:0=4d}.png'.format(Index)
    
    save_image(denorm(FakeImages), os.path.join(Sample_Dir, Fake_Fname), nrow=8)
    
    print('Saving the images....', Fake_Fname)
    
    if show:

        figure, axis = plt.subplots(figsize=(10, 10))
      
        axis.set_xticks([]); axis.set_yticks([])
      
        axis.imshow(make_grid(Fake_Images.cpu().detach(), nrow=4).permute(1, 2, 0))

# save the first image from generator training(its output)
Fixed_Latent = torch.randn(16, LatentSize, 1, 1, device=Device)

Save_Samples(0, Fixed_Latent)

In [ ]:
# lets traing the disc. and generator in parallel for each batch of training data by calling fit method
from tqdm.notebook import tqdm
import torch.nn.functional as F

def Fit(Epochs, LR, Start_idx=1):
    torch.cuda.empty_cache()
    
    # array to store final scores
    Losses_g = []
    Losses_d = []
    Real_scores = []
    Fake_scores = []
    
    # Optimizers for disc. and generator networks
    Opt_d = torch.optim.Adam(Disc.parameters(), lr=LR, betas=(0.5, 0.999))
    Opt_g = torch.optim.Adam(Gene.parameters(), lr=LR, betas=(0.5, 0.999))
    
    for Epoch in range(Epochs):
        for Real_images, _ in tqdm(trainDL):
            
            # start Training discriminator netowkr
            Loss_d, Real_score, Fake_score = Train_Disc(Real_images, Opt_d)
            # start Training generator netowkr
            Loss_g = Train_Gene(Opt_g)
            
        # Store the loss and real scores for generator and disc. network
        Losses_g.append(Loss_g)
        Losses_d.append(Loss_d)
        Real_scores.append(Real_score)
        Fake_scores.append(Fake_score)
        
        # logging and scoring for last batches
        print("Epoch ------> [{}/{}], Loss_gene: {:.4f}, Loss_disc: {:.4f}, Real_score: {:.4f}, Fake_score: {:.4f}".format(
            Epoch+1, Epochs, Loss_g, Loss_d, Real_score, Fake_score))
    
        # Save the output images 
        Save_Samples(Epoch + Start_idx, Fixed_Latent, show=False)
    
    # return their values
    return Losses_g, Losses_d, Real_scores, Fake_scores

# set the learning rate to 0.0002 and train for 1 epoch
LR = 0.0002
Epochs = 25

# start training
History = Fit(Epochs, LR)

## Evaluate The Model

In [ ]:
# assign the values to the set
Losses_G, Losses_D, Real_Scores, Fake_Scores = History

# show the predictions of the AC-GAN model
from IPython.display import Image

Image('./Generated Images/Generated_Images-0001.png')


import cv2
import os

In [ ]:
plt.plot(Real_Scores, '--')
plt.plot(Fake_Scores, '--')
plt.xlabel('Epoch')
plt.ylabel('Score')
plt.legend(['Real', 'Fake'])
plt.title('Scores Plot');

In [ ]:
# check the loss for generator and disc. networks 
plt.plot(Losses_D, '--')
plt.plot(Losses_G, '--')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Discriminator Network', 'Generator Network'])
plt.title('Losses Plot');


In [ ]:
from IPython.display import Image

Image('./Generated Images/Generated_Images-0001.png')
Image('./Generated Images/Generated_Images-0005.png')
Image('./Generated Images/Generated_Images-00010.png')
Image('./Generated Images/Generated_Images-00020.png')
Image('./Generated Images/Generated_Images-00025.png')

# Save the model checkpoints and weights for future references
torch.save(Gene.state_dict(), 'Generator.pth')
torch.save(Disc.state_dict(), 'Discriminator.pth')

## Thank you!